In [22]:
import glob

# Invasive category
invasive_dirs = [
    r'D:\VeligerData\Baylor 2022-03-21_2\Veligers',
    r'D:\VeligerData\invasive',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Pedi-Zebra Veligers',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1a To Baylor\Preserved Zebra Ped 1a To Baylor\Sorted Images\Preserved Zebra Ped 1a',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Zebra Pediveliger Image1a\Zebra Pediveligers',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Umbonal',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Umbonal',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Zebra D-Hinge',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Umbonal'
]

# Non-Invasive category
non_invasive_dirs = [
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Not',
    r'D:\VeligerData\Baylor 2022-03-21_2\NonVeligers\Images_001',
    r'D:\VeligerData\noninvasive',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Not',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Not',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Not',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Not',
    r'D:\VeligerData\Ostracods Day 2 Image1 To Baylor\Not',
    r'D:\VeligerData\Ostracods Day 2 Image2 To Baylor\Not',
    r'D:\VeligerData\Ostracods Day 2 Image12 To Baylor_3\Ostracods Day 2 Image12 To Baylor\Sorted Images\Not',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Not Veligers\O1',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Not',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Not'
]

# Ostracod category
ostracod_dirs = [
    r'D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Preserve Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Preserved Ostracods 1a',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Ostracod Image1\Ostracods1',
    r'D:\VeligerData\Ostracods Day 2 Image1 To Baylor\Ostracods',
    r'D:\VeligerData\Ostracods Day 2 Image3 To Baylor_2\Ostracods Day 2 Image3 To Baylor\Sorted Images\Ostracods',
    r'D:\VeligerData\Ostracods Day 2 Image12 To Baylor_3\Ostracods Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'D:\VeligerData\Ostracods Day 2 Image2 To Baylor\Ostracods',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Ostracod'
]

# List to store subdirectories
invasive_subdirs = []
non_invasive_subdirs = []
ostracod_subdirs = []

# Collect subdirectories in the invasive category
for invasive_dir in invasive_dirs:
    invasive_subdirs.extend(glob.glob(invasive_dir))

# Collect subdirectories in the non-invasive category
for non_invasive_dir in non_invasive_dirs:
    non_invasive_subdirs.extend(glob.glob(non_invasive_dir))
    
for ostracod_dir in ostracod_dirs:
    ostracod_subdirs.extend(glob.glob(ostracod_dir))

In [23]:
ostracod_subdirs

['D:\\VeligerData\\Ostracod Day 2 Image12 Short To Baylor\\Ostracod Day 2 Image12 To Baylor\\Sorted Images\\Ostracods',
 'D:\\VeligerData\\Ostracod vs Pedi-Veliger Examples\\Ostracod vs Pedi-Veliger Examples\\Preserved Ostracods 1 To Baylor\\Preserved Ostracods 1 To Baylor\\Sorted Images\\Preserve Ostracods',
 'D:\\VeligerData\\Ostracod vs Pedi-Veliger Examples\\Ostracod vs Pedi-Veliger Examples\\Preserved Ostracods 1a To Baylor\\Preserved Ostracods 1a To Baylor\\Sorted Images\\Preserved Ostracods 1a',
 'D:\\VeligerData\\To Baylor 2023-01-30\\To Baylor 2023-01-30\\Ostracod Image1\\Ostracods1',
 'D:\\VeligerData\\Ostracods Day 2 Image1 To Baylor\\Ostracods',
 'D:\\VeligerData\\Ostracods Day 2 Image3 To Baylor_2\\Ostracods Day 2 Image3 To Baylor\\Sorted Images\\Ostracods',
 'D:\\VeligerData\\Ostracods Day 2 Image12 To Baylor_3\\Ostracods Day 2 Image12 To Baylor\\Sorted Images\\Ostracods',
 'D:\\VeligerData\\Ostracods Day 2 Image2 To Baylor\\Ostracods',
 'D:\\VeligerData\\USGS Labled Zebr

In [38]:
import glob

# List of directories in x
x = non_invasive_subdirs

# List to store subdirectories
y1 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y1.extend(subdirectories)


In [31]:
import glob

# List of directories in x
x = invasive_subdirs

# List to store subdirectories
y2 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y2.extend(subdirectories)


In [32]:
import glob

# List of directories in x
x = ostracod_subdirs

# List to store subdirectories
y3 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y3.extend(subdirectories)


In [83]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image

def preprocess_images(y1, label_num, target_size=(40, 40)):
    # List to store image files
    image_files = []
    # List to store labels
    labels = []

    # Retrieve image files and create labels for each directory in y1
    for directory in y1:
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file has an image extension
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    # Add the file path to the image_files list
                    image_files.append(os.path.join(root, file))
                    # Add the label to the labels list
                    

    # List to store preprocessed images
    images = []

    # Preprocess each image
    for file in image_files:
        try:
            # Read the image using PIL
            image = Image.open(file)
            # Resize the image using tf.image.resize_with_crop_or_pad()
            image = tf.image.resize_with_crop_or_pad(
                tf.keras.preprocessing.image.img_to_array(image),
                target_size[0],
                target_size[1]
            )
            # Normalize the image pixels for ML training
            image = image / 255.0
            # Add the preprocessed image to the images list
            images.append(image)
            labels.append(label_num)
        except Exception as e:
            print(f"Error processing image: {file}. Skipping...")
            continue

    # Convert the images and labels lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels


In [53]:
X0,Y0 = preprocess_images(y1, label_num = 0)

In [54]:
X1,Y1 = preprocess_images(y2, label_num = 1)

In [84]:
X2,Y2 = preprocess_images(y3, label_num = 2)

Error processing image: D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods\Object_002\._Image_032.png. Skipping...
Error processing image: D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods\Object_002\._Image_036.png. Skipping...


In [86]:
Y2.shape

(4865,)

In [87]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Concatenate the image arrays
train_images = np.concatenate((X0, X1, X2), axis=0)

# Combine the label arrays
train_labels = np.concatenate((Y0, Y1, Y2), axis=0)

# Convert the labels to categorical
train_labels_categorical = to_categorical(train_labels)

# Set the dtype of train_images to float32
train_images = train_images.astype('float32')



In [148]:
# Split the data into train and test sets
from sklearn.utils import shuffle

# Assuming you have the original train_images and train_labels_categorical

# Shuffle the data
train_images, train_labels_categorical = shuffle(train_images, train_labels_categorical)

# Now the shuffled data is assigned to the same variable names

X_train, X_test, Y_train, Y_test = train_test_split(train_images, train_labels_categorical, test_size=0.1, random_state=42)

In [149]:
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    tn = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true) * (y_pred), 0, 1)))
    fn = K.sum(K.round(K.clip((y_true) * (1-y_pred), 0, 1)))
    

    f1_val = tp / ( tp + ( (1/2) * (fp+fn) ) + K.epsilon())
    return f1_val

(42055, 40, 40, 3)

In [150]:
class_weights = np.zeros(3)

unique_labels, counts = np.unique(train_labels, return_counts=True)

total_samples = np.sum(counts)

for i, label in enumerate(unique_labels):
    class_weights[label] = total_samples / (len(unique_labels) * counts[i])

print("Class weights:", class_weights)


Class weights: [ 0.44761988  1.45657671 12.59122987]


In [279]:
class_weights = [.45,1.5,1]

In [281]:
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(40, 40, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(10))
model.add(layers.Dense(3, activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 38, 38, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 19, 19, 32)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 17, 17, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 6, 6, 16)          4624      
                                                                 
 flatten_4 (Flatten)         (None, 576)              

In [153]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=([get_f1]))
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.1, verbose=1, class_weight=class_weights_dict)

Train on 148852 samples, validate on 16540 samples
Epoch 1/100
148672/148852 [============================>.] - ETA: 0s - loss: 0.1782 - get_f1: 0.9104

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


148852/148852 [==============================] - 43s 286us/sample - loss: 0.1781 - get_f1: 0.9105 - val_loss: 0.1542 - val_get_f1: 0.9300
Epoch 2/100
148852/148852 [==============================] - 42s 282us/sample - loss: 0.1393 - get_f1: 0.9308 - val_loss: 0.1215 - val_get_f1: 0.9382
Epoch 3/100
 36000/148852 [======>.......................] - ETA: 24s - loss: 0.1285 - get_f1: 0.9359

In [266]:
pred = model.predict(X_test)
p = np.round(pred)

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [206]:
p.shape

(2000, 3)

In [267]:
res = []
for i in p:
    if(i[0]==1):
        res.append(0)
    elif(i[1]==1):
        res.append(1)
    else:
        res.append(2)

In [268]:
test = []
for i in Y_test:
    if(i[0]==1):
        test.append(0)
    elif(i[1]==1):
        test.append(1)
    else:
        test.append(2)


In [269]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test, res)
print("Accuracy:", accuracy)


Accuracy: 0.9510801545410024


In [271]:
total_samples = len(test)
correct_predictions = 0
class_1_count = 0
num = 1
for i in range(total_samples):
    if test[i] == num:
        class_1_count += 1
        if res[i] == test[i]:
            correct_predictions += 1

class_1_accuracy = correct_predictions / class_1_count
print("Accuracy for Class 1:", class_1_accuracy)


Accuracy for Class 1: 0.9424342105263158


In [277]:
# Define the image path
image_path = r"D:\VeligerData\Baylor 2022-03-21_2\Veligers\Object_031\Image_009.png"
# Read the image using PIL
image = Image.open(image_path)
# Define the target size for resizing
target_size = (40, 40)
# Resize the image using TensorFlow's resize_with_crop_or_pad function
resized_image = tf.image.resize_with_crop_or_pad( tf.keras.preprocessing.image.img_to_array(image), target_size[0],   target_size[1]
)
# Expand the dimensions to match the model's input shape
expanded_image = tf.expand_dims(resized_image, axis=0)
# Evaluate the TensorFlow tensor and obtain the NumPy array
resized_array = tf.keras.backend.eval(expanded_image)
normalized_image = resized_array / 255.0
x = model.predict(normalized_image)
x

array([[1.4065764e-02, 9.8593384e-01, 3.8500878e-07]], dtype=float32)

In [221]:
x

array([[7.2788095e-01, 2.7207914e-01, 3.9928880e-05]], dtype=float32)

In [224]:
total_samples = len(res)
correct_predictions = 0
class_1_count = 0
num = 2
for i in range(total_samples):
    if test[i] == num:
        print(i)

6
11
34
55
58
71
83
162
182
324
359
375
389
410
456
479
507
509
520
602
610
642
644
668
715
734
809
827
838
842
853
882
896
908
944
945
1058
1065
1157
1166
1171
1175
1186
1277
1280
1352
1466
1541
1556
1558
1561
1581
1598
1611
1682
1715
1740
1789
1790
1812
1851
1881
1889
1940
1948
1977
2068
2129
2141
2182
2195
2255
2264
2312
2357
2394
2417
2530
2539
2562
2580
2601
2605
2613
2659
2825
2843
2904
2954
3046
3063
3126
3148
3152
3185
3196
3197
3220
3286
3292
3312
3361
3382
3425
3428
3446
3451
3478
3482
3501
3515
3566
3579
3601
3710
3722
3727
3753
3757
3777
3788
3809
3827
3881
3889
3897
3927
3961
3992
4038
4049
4062
4218
4361
4392
4404
4479
4554
4570
4590
4810
4887
4942
4955
4987


In [184]:
res[25]

1

In [ ]:
from tensorflow.keras.models import load_model
# Assuming you have a Keras model named "model"
model.save("single_image_model.h5")